In [ ]:
import sys
import os
import argparse
import numpy as np
import scipy.signal as sp
import processing as pr
import matplotlib.pyplot as plt
import pandas as pd
from ruamel.yaml import YAML as ym
import datetime
import copy
import re

# Use this for interactive graphs - nice for exploring
%matplotlib widget

#%matplotlib inline

import mplcursors

sys.path.append("../preprocessing")
from generate_chirp import generate_chirp

# Widgets are only needed if you want to use the interactive plot at the end
# Installation instructions: https://ipywidgets.readthedocs.io/en/latest/user_install.html
import ipywidgets as widgets

import scipy.fft

In [ ]:
plt.close('all')

In [ ]:
# 2021-11-24 Roble Pre-Test
# prefix = "../../drone/radar_data/20211124-roble/20211124_173951"
# px4_logs = None

# 2021-11-24 Flight 1
# prefix = "../../drone/radar_data/20211124-lag/20211124_185351"
# px4_logs = "../../px4_logs/2021-11-24/18_45_08/18_45_08_"
# telemetry_time_offset_s = 55

# 2021-11-24 Flight 2
# prefix = "../../drone/radar_data/20211124-lag/20211124_192115"
# px4_logs = "../../px4_logs/2021-11-24/19_14_06/19_14_06_"
# telmetry_time_offset_s = 34

# 2021-12-01 Roble Pretest
# prefix = "../../drone/radar_data/20211201-roble-pretest/20211201_184957"
# px4_logs = None

# 2021-12-01 Lag Flight
# prefix = "../../drone/radar_data/20211201-lag/20211201_200316"
# px4_logs = "../../px4_logs/2021-12-01/19_57_33/19_57_33_"
# telemetry_time_offset_s = 20

# 2021-12-06 Roble
# prefix = "../../drone/radar_data/20211206-roble/20211206_183918" # Moving test 1
# px4_logs = None

#prefix = "../../drone/radar_data/20211206-roble/20211206_192749"
#px4_logs = None

# 2021-12-08 Lake Lag Flight
# prefix = "../../drone/radar_data/20211208-lag/20211208_193107"
# px4_logs = "../../px4_logs/2021-12-08/19_17_22/19_17_22_"
# telemetry_time_offset_s = 175


# prefix = "../../drone/radar_data/20220308-aeromao-field/20220307_153106" # 2nd flight on 3/8 w/ Aeromao

# Svalbard Day 2

#prefix = "../../drone/radar_data/20220319-tellbreen/20220319_061201"
prefix = "../../drone/radar_data/20220319-tellbreen/20220319_070117" # original spot, just before separating wings
#prefix = "../../drone/radar_data/20220319-tellbreen/20220319_070340" # time domain looks normal
#prefix = "../../drone/radar_data/20220319-tellbreen/20220319_073702" # time domain starts looking weird
#prefix = "../../drone/radar_data/20220319-tellbreen/20220319_074151" # high power, wings separated
#prefix = "../../drone/radar_data/20220319-tellbreen/20220319_081044" # moved to location 2 - near Anna - wings still separated
#prefix = "../../drone/radar_data/20220319-tellbreen/20220319_082104" # antennas back on UAV
#prefix = "../../drone/radar_data/20220319-tellbreen/20220319_082241" # gain tuned down a bit
#prefix = "../../drone/radar_data/20220319-tellbreen/20220319_083328" # rectangular window, settings used for 1st walk
prefix = "../../drone/radar_data/20220319-tellbreen/20220319_083855" # transect w/ the above settings
#prefix = "../../drone/radar_data/20220319-tellbreen/20220319_084726"

# Svalbard Day 3

prefix = "../../drone/radar_data/20220320-tellbreen/20220320_063039" # spot 1 with Anna's antennas
#prefix = "../../drone/radar_data/20220320-tellbreen/20220320_063632" # same but higher TX power and 20 MHz bandwidth (still Anna's antennas)
#prefix = "../../drone/radar_data/20220320-tellbreen/20220320_064106" # rectangular window
#prefix = "../../drone/radar_data/20220320-tellbreen/20220320_064503" # rect window, longer chirp (100 us)
#prefix = "../../drone/radar_data/20220320-tellbreen/20220320_070536" # first one with UAV antennas
#prefix = "../../drone/radar_data/20220320-tellbreen/20220320_071226" # nice clear reflection at 68 meters (almost certainly not real)
#prefix = "../../drone/radar_data/20220320-tellbreen/20220320_074049"
#prefix = "../../drone/radar_data/20220320-tellbreen/20220320_075149" # transect - but clipping :(
#prefix = "../../drone/radar_data/20220320-tellbreen/20220320_080259" # spot 2, good candidate at 73 meters
#prefix = "../../drone/radar_data/20220320-tellbreen/20220320_084343" # transect - no gps recording but presumably spot 2 to spot 1
#prefix = "../../drone/radar_data/20220320-tellbreen/20220320_084646" # transect - spot 1 to spot 2 (caltopo recording "3/20/22")
#prefix = "../../drone/radar_data/20220320-tellbreen/20220320_090318" # transect - spot 2 to spot 1 (caltopo recording "3/20/22 2")
#prefix = "../../drone/radar_data/20220320-tellbreen/20220320_094509" # final transect (spot 2 back to hut)

# Svalbard Day 3 - Anna's Data
#prefix = "/home/thomas/Documents/StanfordGrad/RadioGlaciology/svalbard_2022_anna_data/03202022_anna/20220320_142444" # spot 1 (I think) ~51 m
#prefix = "/home/thomas/Documents/StanfordGrad/RadioGlaciology/svalbard_2022_anna_data/03202022_anna/20220320_162821" # spot 2 (I think) ~68 m

# Svalbard Day 4
#prefix = "../../drone/radar_data/20220321-tellbreen/20220321_051828" # likely at 28 and 82 - similar settings to yesterday, just tweaked gain
#prefix = "../../drone/radar_data/20220321-tellbreen/20220321_052716" # checking what rectangular window looks like - seems different from yeterday
#prefix = "../../drone/radar_data/20220321-tellbreen/20220321_053033" # moved snowmobile away and repeated
#prefix = "../../drone/radar_data/20220321-tellbreen/20220321_053254" # 5mhz bandwidth, rect window
#prefix = "../../drone/radar_data/20220321-tellbreen/20220321_053539" # 10 mhz, blackman
#prefix = "../../drone/radar_data/20220321-tellbreen/20220321_054145" # shorter 50 us chirp
#prefix = "../../drone/radar_data/20220321-tellbreen/20220321_054855" # more antenna separation
#prefix = "../../drone/radar_data/20220321-tellbreen/20220321_055136" # 0dbm transmit pwr
#prefix = "../../drone/radar_data/20220321-tellbreen/20220321_055340" # back to 56mhz bandwidth
#prefix = "../../drone/radar_data/20220321-tellbreen/20220321_055700" # 10 dbm tx - looks good at 76m
#prefix = "../../drone/radar_data/20220321-tellbreen/20220321_060029" # 20 mhz bandwidth
#prefix = "../../drone/radar_data/20220321-tellbreen/20220321_060814" # plugged in LNA
#prefix = "../../drone/radar_data/20220321-tellbreen/20220321_061013" # lower rx gain
#prefix = "../../drone/radar_data/20220321-tellbreen/20490108_162932" # added PA and dropped tx gain by 30 db
#prefix = "../../drone/radar_data/20220321-tellbreen/20490108_163155" # gain tuning - looks good at 74 m
#prefix = "../../drone/radar_data/20220321-tellbreen/20490108_163612" # 30 mhz bandwidth, 100 us chirp, direct path 60 db, reflection 17.4 db
#prefix = "../../drone/radar_data/20220321-tellbreen/20490108_163832" # 200 us chirp to see snr change? - dp 66 db, ref 23
#prefix = "../../drone/radar_data/20220321-tellbreen/20490108_164132" # 30 us chirp 49 db and 5 db
#prefix = "../../drone/radar_data/20220321-tellbreen/20490108_164553" # 500 us 
#prefix = "../../drone/radar_data/20220321-tellbreen/20490108_164916" # drop gain 5 db
#prefix = "../../drone/radar_data/20220321-tellbreen/20490108_165158" # move antennas closer - something weird with rx chirp
#prefix = "../../drone/radar_data/20220321-tellbreen/20490108_165620" # retry
#prefix = "../../drone/radar_data/20220321-tellbreen/20490108_165809" # drop 5 db lower on tx gain
#prefix = "../../drone/radar_data/20220321-tellbreen/20490108_165951" # 5db higher instead -- look into this later
#prefix = "../../drone/radar_data/20220321-tellbreen/20490108_170355" # 15 mhz bandwidth (halved from 30)
#prefix = "../../drone/radar_data/20220321-tellbreen/20490108_170745" # moved antennas closer
#prefix = "../../drone/radar_data/20220321-tellbreen/20490108_170920" # tx gain tune
#prefix = "../../drone/radar_data/20220321-tellbreen/20490108_171110" # 380 to 390 mhz center freq
#prefix = "../../drone/radar_data/20220321-tellbreen/20490108_171743" # 30 mhz band at 400 center#
#prefix = "../../drone/radar_data/20220321-tellbreen/20490108_172200" # further apart again
#prefix = "../../drone/radar_data/20220321-tellbreen/20490108_172418" #more tx gain
#prefix = "../../drone/radar_data/20220321-tellbreen/20490108_172624" # 380 mhz center
#prefix = "../../drone/radar_data/20220321-tellbreen/20490108_173128" # lower tx power
#prefix = "../../drone/radar_data/20220321-tellbreen/20490108_173320" # 50 mhz bw
#prefix = "../../drone/radar_data/20220321-tellbreen/20490108_174058"

# Pre-iceland testing
#prefix = "../../drone/radar_data/20220518-roble/20220518_091525" # first walk (that didn't mess up)
#prefix = "../../drone/radar_data/20220518-roble/20220518_093007" # second walk

# 5/23 Lake Lag Flights
#prefix = "../../drone/radar_data/20220523-lag/20220523_110123" # Lake Lag 5/23 Flight 1 -- warning: extremely large file - make sure you only open the first 6-10 minutes of it
#px4_logs = "../../px4_logs/2022-05-23/17_39_45/17_39_45_"
#telemetry_time_offset_s = -87
# Plotting settings for above
# n_stack = 15
# start_time_s = 200
# duration_s = 210 # -1 for entire thing
# use_recorded_chirp = False

# prefix = "../../drone/radar_data/20220523-lag/20220523_121145" # Lake Lag 5/23 Flight 2
# px4_logs = "../../px4_logs/2022-05-23/19_02_51/19_02_51_"
# telemetry_time_offset_s = -53
# Plotting settings for above
# n_stack = 15
# start_time_s = 200
# duration_s = 150 # -1 for entire thing
# use_recorded_chirp = False

# 5-25 Lag
#prefix = "../../drone/radar_data/20220525-lag/20220525_110959" # Lake Lag 5/25 Flight 1
#prefix = "../../drone/radar_data/20220525-lag/20220525_113900" # On the ground, right after a reboot - same settings, very different result
prefix = "../../drone/radar_data/20220525-lag/20220525_115609" # Lake Lag 5/25 Flight 2

# Vatnajokull Day 1
#prefix = "../../drone/radar_data/20220602-vatnajokull/20220602_081600" # first measurement, maybe something ~50 m
#prefix = "../../drone/radar_data/20220602-vatnajokull/20220602_082523" # higher RX gain
#prefix = "../../drone/radar_data/20220602-vatnajokull/20220602_083026" # 20 mhz bandwidth
#prefix = "../../drone/radar_data/20220602-vatnajokull/20220602_083548" # longer pulse (200us) and rect window - something wrong with data?
#prefix = "../../drone/radar_data/20220602-vatnajokull/20220602_084054" # back to blackman window, changed center freq to 350
#prefix = "../../drone/radar_data/20220602-vatnajokull/20220602_084809" # fixed RX length
#prefix = "../../drone/radar_data/20220602-vatnajokull/20220602_085640" # rect window again (350 mhz center)
#prefix = "../../drone/radar_data/20220602-vatnajokull/20220602_090544" # 340 mhz center
#prefix = "../../drone/radar_data/20220602-vatnajokull/20220602_090738" # 320 mhz center i think
#prefix = "../../drone/radar_data/20220602-vatnajokull/20220602_090947" # 380 mhz
#prefix = "../../drone/radar_data/20220602-vatnajokull/20220602_094110" # 420 mhz
#prefix = "../../drone/radar_data/20220602-vatnajokull/20220602_094340" # 400 mhz, blackman again -- good looking one
#prefix = "../../drone/radar_data/20220602-vatnajokull/20220602_094602" # 10 presum, testing manager
#prefix = "../../drone/radar_data/20220602-vatnajokull/20220602_094828" # 1 presum
#prefix = "../../drone/radar_data/20220602-vatnajokull/20220602_095225" # same thing
#prefix = "../../drone/radar_data/20220602-vatnajokull/20220602_095431" # more tx power
#prefix = "../../drone/radar_data/20220602-vatnajokull/20220602_095733" # gain tweaking
#prefix = "../../drone/radar_data/20220602-vatnajokull/20220602_100358" # walk

# Vatnajokull Day 2
#prefix = "../../drone/radar_data/20220603-vatnajokull/20220603_050301" # first test, slightly longer PRF because of late command errors
# prefix = "../../drone/radar_data/20220603-vatnajokull/20220603_050555" # dropped tx gain to 70 db bc of saturation
# prefix = "../../drone/radar_data/20220603-vatnajokull/20220603_050813" # dropped tx to 65 db
# prefix = "../../drone/radar_data/20220603-vatnajokull/20220603_051239" # 3800 mhz center (accidentally)
# prefix = "../../drone/radar_data/20220603-vatnajokull/20220603_051455" # 380 mhz center - looks good? deeper targets moving a lot though
# prefix = "../../drone/radar_data/20220603-vatnajokull/20220603_051826" # same but w/ manager
# prefix = "../../drone/radar_data/20220603-vatnajokull/20220603_052708" # walk 1 (no caltopo)
# prefix = "../../drone/radar_data/20220603-vatnajokull/20220603_054723" # ground, picked up, ground - all same place
# prefix = "../../drone/radar_data/20220603-vatnajokull/20220603_055214" # wobbled on the ground, then still
# prefix = "../../drone/radar_data/20220603-vatnajokull/20220603_055656" # wobbled then held still
# prefix = "../../drone/radar_data/20220603-vatnajokull/20220603_073718" # retry after snowmobile break
# prefix = "../../drone/radar_data/20220603-vatnajokull/20220603_074036" # backed off RX gain by 2 db and changed center to 390 mhz
#prefix = "../../drone/radar_data/20220603-vatnajokull/20220603_074322" # ground then picked up
#prefix = "../../drone/radar_data/20220603-vatnajokull/20220603_083823" # FLIGHT!!! (and crash :( ))

# Vatnajokull Day 3 - Hut - Anna's Data
#prefix = "../../drone/radar_data/anna-or-pres/20220606_anna/20220606_193318"
#prefix = "../../drone/radar_data/anna-or-pres/20220320_anna/20220320_163613" # Test data from Svalbard

# Vatnajokull Day 4
#prefix = "../../drone/radar_data/20220607-vatnajokull-day4/20220607_054356" # ground test with 380 mhz center, 50 us chirp
#prefix = "../../drone/radar_data/20220607-vatnajokull-day4/20220607_054743" # increased chirp length to 80 us (same as RX duration, oops)
#prefix = "../../drone/radar_data/20220607-vatnajokull-day4/20220607_055048" # 70 us
#prefix = "../../drone/radar_data/20220607-vatnajokull-day4/20220607_055406" # button manager
#prefix = "../../drone/radar_data/20220607-vatnajokull-day4/20220607_055915" # manager, 60 us
prefix = "../../drone/radar_data/20220607-vatnajokull-day4/20220607_065057" # Flight!!!

# Vatnajokull Day 5
#prefix = "../../drone/radar_data/20220608-vatnajokull/20220608_060135" # ground test
#prefix = "../../drone/radar_data/20220608-vatnajokull/20220608_064444" # Flight (with radar)

px4_logs = None



yaml_file = prefix + "_config.yaml"
bin_file = prefix + "_rx_samps.bin"
log_file = prefix + "_uhd_stdout.log"

# For PX4 logs: They must be converted to a set of CSV files with pyulog
# For example: `ulog2csv 17_39_45.ulg -o 17_39_45/`


n_stack = 1 # Do additional stacking in this notebook - set > 1 to enable
# Thomas's system at 56 MHz sample rate - zero_sample_idx = 159
# Anna's system - 38 I think?
zero_sample_idx = 159 #159 # (6 for anna's system?) # The index of the 0 distance sample -- will change with platforms and config changes -- if unsure, just set to 0

#zero_sample_idx = 160

In [ ]:
# Initialize Constants
yaml = ym()
with open(yaml_file) as stream:
    config = yaml.load(stream)
    sample_rate = config["PLOT"]["sample_rate"]    # Hertz
    sig_speed = config["PLOT"]["sig_speed"] / np.sqrt(3.17) ## TODO ICE

    #expected_n_rxs = int(config['CHIRP']['num_pulses'] / config['CHIRP']['num_presums'])
    
    rx_len_samples = int(config['CHIRP']['rx_duration'] * config['GENERATE']['sample_rate'])
    
rx_samps = bin_file

In [ ]:
errors = None

if os.path.exists(log_file):
    errors = {}
    
    log_f = open(log_file, 'r')
    log = log_f.readlines()
    
    for idx, line in enumerate(log):
        if "Receiver error:" in line:
            error_code = re.search("(?:Receiver error: )([\w_]+)", line).groups()[0]
            chirp_idx = int(re.search("(?:Scheduling chirp )([\d]+)", log[idx-1]).groups()[0])
            errors[chirp_idx] = error_code
            if error_code != "ERROR_CODE_LATE_COMMAND":
                print(f"WARNING: Uncommon error in the log: {error_code} (on chirp {chirp_idx})")
                print(f"Full message: {line}")
else:
    print(f"WARNING: No log file found. This is fine, but checks for error codes will be disabled.")
    print(f"(Looking for a log file in: {log_file})")

In [ ]:
config_no_tx_window = copy.deepcopy(config)
config_no_tx_window['GENERATE']['window'] = 'rectangular'
config_blackman_window = copy.deepcopy(config)
config_blackman_window['GENERATE']['window'] = 'blackman'

In [ ]:
# Read and plot RX/TX
# This cell loads all of the data - it can take a while with a large file. You don't need to re-run this cell if you only change n_stack
_, tx_sig = generate_chirp(config)
_, tx_sig_no_window = generate_chirp(config_no_tx_window)
_, tx_sig_blackman_window = generate_chirp(config_blackman_window)
pr.plotChirpVsTime(tx_sig, 'Transmitted Chirp', sample_rate)
print(f"len(tx_sig): {len(tx_sig)}")

t0 = datetime.datetime.now()

max_chunk_size = int(5e8)
max_seconds_to_load = 60*20
load_start_seconds = 0#200

max_file_size_bytes = rx_len_samples*int(1/config['CHIRP']['pulse_rep_int'])*8*max_seconds_to_load
load_start_bytes = rx_len_samples*int(1/config['CHIRP']['pulse_rep_int'])*8*load_start_seconds

file_size_bytes = os.path.getsize(rx_samps) - load_start_bytes
if file_size_bytes > max_file_size_bytes:
    print(f"WARNING: File is {file_size_bytes/(2**30):.2f} GB ({file_size_bytes / (rx_len_samples*int(1/config['CHIRP']['pulse_rep_int'])*2):.2f} seconds). Only loading the first {max_seconds_to_load} seconds.")
    file_size_bytes = max_file_size_bytes

rx_sig = np.zeros((file_size_bytes//8,), dtype=np.csingle)
for start_offset in np.arange(0, file_size_bytes, max_chunk_size, dtype=np.int64):
    if start_offset + max_chunk_size > file_size_bytes:
        #print(f"{start_offset} (last) count: {file_size_bytes-start_offset}, offset: {load_start_bytes+start_offset}")
        rx_sig[(start_offset//8):] = pr.extractSig(rx_samps, count=file_size_bytes-start_offset, offset=load_start_bytes+start_offset)
    else:
        #print(f"{max_chunk_size}, {start_offset}")
        rx_sig[(start_offset//8):((start_offset//8)+(max_chunk_size//2))] = pr.extractSig(rx_samps, count=max_chunk_size, offset=load_start_bytes+start_offset)

#rx_sig = pr.extractSig(rx_samps, count=rx_len_samples*int(1/config['CHIRP']['pulse_rep_int'])*120*4)
# 13.3 GB peak consumption, 4.5 steady state
# new version: 9.5 GB peak, 4.5 steady state

print(datetime.datetime.now() - t0)

In [ ]:
n_rxs = len(rx_sig) // rx_len_samples
rx_sig_reshaped = np.transpose(np.reshape(rx_sig, (n_rxs, rx_len_samples), order='C'))
errors_removed = False # Keep track of if the cell removing errors from rx_sig_reshaped has already been run

print(f"len(rx_sig): {len(rx_sig)}")
print(f"n_rxs: {n_rxs}")
print(f"rx_sig shape: {np.shape(rx_sig)}")
print(f"rx_sig_reshaped shape: {np.shape(rx_sig_reshaped)}")

#rx_sig_reshaped = rx_sig_reshaped[:,200:]

In [ ]:
# Choose what to do with chirps with a reported error (usually ERROR_CODE_LATE_COMMAND)
# Options are:
# None - do nothing
# 'zeros' - replace with zeros
# 'remove' - remove them from the rx_sig_reshaped array
error_behavior = 'remove'

if errors is None:
    if error_behavior is not None:
        print(f"WARNING: Requested doing something with errors but no log file was loaded. Defaulting to doing nothing.")
elif error_behavior == 'zeros':
    error_idxs = np.array(list(errors.keys()))
    rx_sig_reshaped[:,error_idxs] = 0
elif error_behavior == 'remove':
    if errors_removed:
        print(f"WARNING: Error chirps already removed since rx_sig_reshaped created. Not doing again.")
    else:
        error_idxs = np.array(list(errors.keys()))
        all_idxs = np.arange(rx_sig_reshaped.shape[1])
        keep_idxs = [x for x in all_idxs if x not in error_idxs]

        rx_sig_reshaped = rx_sig_reshaped[:, keep_idxs]
        n_rxs = len(keep_idxs)
        errors_removed = True
    
print(f"n_rxs: {n_rxs}")
print(f"rx_sig_reshaped shape: {np.shape(rx_sig_reshaped)}")

In [ ]:
chirp_idx = 200
fig, axs = pr.plotChirpVsTime(rx_sig_reshaped[:,chirp_idx], 'Recieved Chirp', sample_rate)
axs[0].text(0, 1.1, prefix.split("/")[-1] + "\n" + f"chirp_idx = {chirp_idx}", horizontalalignment='left', verticalalignment='center', transform=axs[0].transAxes)
plt.savefig(prefix + "_rx_chirp.png")

In [ ]:
# RX Chirp
analytic_signal = scipy.signal.hilbert(np.abs(rx_sig_reshaped[zero_sample_idx:,0]))
rx_chirp_amplitude_envelope = np.abs(analytic_signal)[zero_sample_idx:zero_sample_idx+len(tx_sig)]

# TX Chirp
analytic_signal = scipy.signal.hilbert(np.abs(tx_sig))
tx_chirp_amplitude_envelope = np.abs(analytic_signal)

# Filtered RX chirp window
filtered_rx_chirp_amplitude_envelope = scipy.ndimage.median_filter(rx_chirp_amplitude_envelope, size=20)
ext = np.concatenate([[filtered_rx_chirp_amplitude_envelope[0]]*15, filtered_rx_chirp_amplitude_envelope, [filtered_rx_chirp_amplitude_envelope[-1]]*15])
filtered_rx_chirp_amplitude_envelope = np.convolve(ext, np.hamming(31)/np.sum(np.hamming(31)), mode='valid')

tx_chirp_scaling_factor = (1 / filtered_rx_chirp_amplitude_envelope)
tx_chirp_scaling_factor = tx_chirp_scaling_factor / np.max(tx_chirp_scaling_factor)

print(np.shape(filtered_rx_chirp_amplitude_envelope))
print(np.shape(rx_chirp_amplitude_envelope))

fig, ax = plt.subplots()
ax.plot(rx_chirp_amplitude_envelope, label="Actual RX")
ax.plot(filtered_rx_chirp_amplitude_envelope, label="Filtered RX")
ax.plot(tx_chirp_amplitude_envelope, label="TX")
ax.plot(tx_chirp_scaling_factor, label="Scaling Factor")
ax.legend()

print(len(filtered_rx_chirp_amplitude_envelope))
#print([f"{x}" for x in filtered_rx_chirp_amplitude_envelope])

In [ ]:
# def butter_lowpass(cutoff, fs, order=5):
#     return scipy.signal.butter(order, cutoff, fs=fs, btype='low', analog=False)

# def butter_lowpass_filter(data, cutoff, fs, order=5):
#     b, a = butter_lowpass(cutoff, fs, order=order)
#     y = scipy.signal.lfilter(b, a, data)
#     return y

# chirp_scaling = np.cumsum(np.random.randn(len(tx_sig)))
# chirp_scaling = butter_lowpass_filter(chirp_scaling, 0.001, 1)
# chirp_scaling = chirp_scaling - np.min(chirp_scaling)
# chirp_scaling = chirp_scaling / np.max(chirp_scaling)

tx_sig_autocorr = sp.correlate(tx_sig, tx_sig, mode='same', method='auto')
tx_sig_rx_envelope_corr = sp.correlate(tx_sig, tx_sig_no_window*rx_chirp_amplitude_envelope, mode='same', method='auto')
#tx_sig_autocorr = sp.correlate(tx_sig, tx_sig * chirp_scaling, mode='same', method='auto')
rx_sig_autocorr = sp.correlate(rx_sig_reshaped[zero_sample_idx:zero_sample_idx+len(tx_sig),0], rx_sig_reshaped[zero_sample_idx:zero_sample_idx+len(tx_sig),0], mode='same', method='auto')

distance_to_reflector_autocorr = np.linspace(0, len(tx_sig_autocorr)/sample_rate, len(tx_sig_autocorr)) * sig_speed / 2
distance_to_reflector_autocorr = distance_to_reflector_autocorr - distance_to_reflector_autocorr[len(tx_sig_autocorr)//2]

fig, ax = plt.subplots(figsize=(10,5))
tx_sig_autocorr_db = 20*np.log10(np.abs(tx_sig_autocorr))
ax.plot(distance_to_reflector_autocorr, tx_sig_autocorr_db - np.max(tx_sig_autocorr_db), label="TX sig autocorr")
ax.plot(distance_to_reflector_autocorr, 20*np.log10(tx_sig_rx_envelope_corr) - np.max(tx_sig_autocorr_db), label="TX sig w/ RX envelope scaling corr")
ax.plot(distance_to_reflector_autocorr, 20*np.log10(rx_sig_autocorr) - np.max(tx_sig_autocorr_db), label="RX sig autocorr")

ax.set_xlim(-300,300)
#ax.set_xlim(len(tx_sig_autocorr)/2 - 200, len(tx_sig_autocorr)/2 + 200)
ax.set_ylim(-400,0)
ax.legend()

In [ ]:
# Must enable widgets (see imports block) for this to work

def plot_timestep(start_chirp_idx, n_stack):
    
    stacked = np.mean(rx_sig_reshaped[:,start_chirp_idx:start_chirp_idx+n_stack], axis=1)
    
    xcorr_res_tmp = sp.correlate(stacked, tx_sig_blackman_window, mode='valid', method='direct')
    
    slow_time = np.linspace(0, config['CHIRP']['pulse_rep_int']*config['CHIRP']['num_presums']*n_rxs, np.shape(rx_sig_reshaped)[1])
    
    distance_to_reflector = np.linspace(0, np.shape(xcorr_res_tmp)[0]/sample_rate, np.shape(xcorr_res_tmp)[0]) * sig_speed / 2
    distance_to_reflector = distance_to_reflector - distance_to_reflector[zero_sample_idx]
    
    fig, ax = plt.subplots(figsize=(10,5), facecolor='white')
    line = ax.plot(distance_to_reflector, 20*np.log10(np.abs(xcorr_res_tmp)), label="Data")
    mplcursors.cursor(line)
    #ax.set_ylim(0,100)
    ax.set_ylim(-40,80)
    ax.set_ylim(-20,40)
    #ax.set_ylim(-20,80)
    ax.set_xlim(-20, 120)
    #ax.set_xlim(-20, 700)
    ax.set_xlim(-200,600)
    ax.set_xlim(-10,250)
    #ax.set_xlim(-500,500)
    ax.set_xlabel('Distance [m]')
    ax.set_ylabel('Power [dB]')
    ax.set_title(f't = {slow_time[start_chirp_idx]:.2f} s')
    ax.grid()
    
    direct_path_time_offset = distance_to_reflector[np.argmax(np.abs(xcorr_res_tmp))]
    
    #ax.plot(distance_to_reflector_autocorr + direct_path_time_offset, 20*np.log10(np.abs(tx_sig_rx_envelope_corr)), label="TX sig w/ RX envelope scaling corr", alpha=0.2)
    ax.plot(-1*distance_to_reflector + 2*direct_path_time_offset, 20*np.log10(np.abs(xcorr_res_tmp)), label="Data (distance flipped)", alpha=0.2)
    
    # Theoretical TX sig autocorr
    tx_sig_autocorr = sp.correlate(tx_sig, tx_sig, mode='same', method='auto')
    tx_sig_autocorr_db = 20*np.log10(np.abs(tx_sig_autocorr))

    distance_to_reflector_autocorr = np.linspace(0, len(tx_sig_autocorr)/sample_rate, len(tx_sig_autocorr)) * sig_speed / 2
    distance_to_reflector_autocorr = distance_to_reflector_autocorr - distance_to_reflector_autocorr[len(tx_sig_autocorr)//2]

    #ax.plot(distance_to_reflector_autocorr, tx_sig_autocorr_db - np.max(tx_sig_autocorr_db) + np.max(20*np.log10(np.abs(xcorr_res_tmp))), label="Theoretical TX sig autocorr")
    
    rx_sig_autocorr_peak_idx = np.argmax(np.abs(rx_sig_autocorr))
    xcorr_peak_idx = np.argmax(np.abs(xcorr_res_tmp))
    print(xcorr_peak_idx)
    rx_sig_autocorr_aligned = rx_sig_autocorr[rx_sig_autocorr_peak_idx - xcorr_peak_idx : rx_sig_autocorr_peak_idx + (len(xcorr_res_tmp) - xcorr_peak_idx)]
    rx_sig_autocorr_aligned = rx_sig_autocorr_aligned  * (np.max(np.abs(xcorr_res_tmp)) / np.max(np.abs(rx_sig_autocorr_aligned))) # scale peak

    #ax.plot(distance_to_reflector, 20*np.log10(rx_sig_autocorr_aligned))
    
    #ax.plot(distance_to_reflector, 20*np.log10(np.maximum(np.abs(xcorr_res_tmp) - 0.95*np.abs(rx_sig_autocorr_aligned), 0.1)))
    direct_path_sample_idx = np.argmax(np.abs(xcorr_res_tmp))
    xcorr_subtracted = (np.abs(xcorr_res_tmp))
    xcorr_subtracted[direct_path_sample_idx:2*direct_path_sample_idx] -= (np.abs(xcorr_res_tmp[direct_path_sample_idx:0:-1]))

    #np.abs(xcorr_res_tmp[:-(direct_path_sample_offset)]) - 
    #ax.plot(distance_to_reflector, 20*np.log10(np.abs(xcorr_subtracted)))
    
    #ax.plot(distance_to_reflector_autocorr + direct_path_time_offset, 20*np.log10(np.abs(rx_sig_autocorr_mean)))
    
    ax.legend()
    
    ax.text(0, 1.05, prefix.split("/")[-1] + "\n" + f"n_stack = {n_stack}, start_chirp_idx = {start_chirp_idx}", horizontalalignment='left', verticalalignment='center', transform=ax.transAxes)
    
    fig.show()
    
    return fig, ax
    
widgets.interact(plot_timestep,
                 start_chirp_idx=widgets.IntSlider(min=0, max=np.shape(rx_sig_reshaped)[1]-1, step=1, value=0),
                 n_stack=widgets.IntSlider(min=1, max=100, step=1, value=0))

In [ ]:
## Figure version

n_stack = 30
start_time_s = 0 #100
duration_s = -1 #400 # -1 for entire thing
use_recorded_chirp = False
    
slow_time = np.linspace(0, config['CHIRP']['pulse_rep_int']*config['CHIRP']['num_presums']*n_rxs, np.shape(rx_sig_reshaped)[1])
if duration_s < 0:
    duration_s = np.max(slow_time) - start_time_s

start_idx = np.argmin(np.abs(slow_time - start_time_s))
end_idx = np.argmin(np.abs(slow_time - (start_time_s + duration_s)))
actual_duration_s = slow_time[end_idx] - slow_time[start_idx]

rx_sig_cropped = rx_sig_reshaped[:, start_idx:end_idx]

# Coherent stacking
xcorr_results = np.zeros(((rx_len_samples-len(tx_sig))+1, np.shape(rx_sig_cropped)[1]//n_stack), dtype=np.csingle)
#print(f"shape of xcorr_results: {np.shape(xcorr_results)}")

for res_idx in range(np.shape(xcorr_results)[1]):
    stacked = np.mean(rx_sig_cropped[:,res_idx*n_stack:(res_idx+1)*n_stack], axis=1)
    if use_recorded_chirp:
        xcorr_results[:, res_idx] = sp.correlate(stacked, rx_sig_reshaped[:len(tx_sig),res_idx*n_stack], mode='valid', method='auto')
    else:
        xcorr_results[:, res_idx] = sp.correlate(stacked, tx_sig_blackman_window, mode='valid', method='auto')
    

        
fig, ax = plt.subplots(1,1, figsize=(10,5), facecolor='white', sharex=True)
axs = [ax]

slow_time = np.linspace(start_time_s, start_time_s+actual_duration_s, np.shape(xcorr_results)[1])
distance_to_reflector = np.linspace(0, np.shape(xcorr_results)[0]/sample_rate, np.shape(xcorr_results)[0]) * sig_speed / 2
if not use_recorded_chirp:
    distance_to_reflector = distance_to_reflector - distance_to_reflector[zero_sample_idx]

p = axs[0].pcolormesh(slow_time, distance_to_reflector, 20*np.log10(np.abs(xcorr_results)), shading='auto', cmap='inferno', vmin=-5, vmax=20)
clb = fig.colorbar(p, ax=axs[0])
clb.set_label('Power [dB]')
axs[0].set_xlabel('Time [s]')
axs[0].set_ylabel('Distance to reflector [m]')

# Plot PX4 log data

if px4_logs is not None:
    #df = pd.read_csv(px4_logs + "distance_sensor_0.csv")
    #axs[0].plot(((df['timestamp']-np.min(df['timestamp']))*1e-6)-telemetry_time_offset_s, df['current_distance'], label='Laser Rangefinder', linestyle=(0, (5,10)))
    df = pd.read_csv(px4_logs + "estimator_global_position_0.csv")
    axs[0].plot(((df['timestamp']-np.min(df['timestamp']))*1e-6)-telemetry_time_offset_s, df['alt']-np.min(df['alt']), label='UAV Altitude',
                linestyle=(0, (5,5)), linewidth=2, color='white', alpha=0.9)
    axs[0].legend()
    
# # Plot error times
# if errors is not None:
#     error_idxs = np.array([(x-start_idx)//n_stack for x in list(errors.keys())])
#     error_idxs[error_idxs >= len(slow_time)] = len(slow_time)-1
#     error_y_loc = np.zeros((len(error_idxs),))
#     axs[0].scatter(slow_time[error_idxs], error_y_loc)


axs[0].set_ylim(100, 0)
#axs[0].set_ylim(100, 0)
#axs[0].set_ylim(80,100)
#axs[0].set_title(f"n_stack = {n_stack}")
axs[0].set_xlim(start_time_s,start_time_s+duration_s)

ax.text(0, 1.05, prefix.split("/")[-1] + "\n" + f"n_stack = {n_stack}", horizontalalignment='left', verticalalignment='center', transform=ax.transAxes)

for item in ([ax.title, ax.xaxis.label, ax.yaxis.label, clb.ax.yaxis.label] +
             ax.get_xticklabels() + ax.get_yticklabels() + clb.ax.get_yticklabels()): #ax.get_legend().get_texts()
    item.set_fontsize(18)
    item.set_fontfamily('sans-serif')
    item.set_fontname('Source Sans Pro')
    
fig.tight_layout()
#fig.savefig('radargram.png', dpi=300)

In [ ]:
def plot_radargram(n_stack, start_time_s=0, duration_s=50): #start_time_s=150, duration_s=250):
    
    overlay_distance_on_radargram = True
    
    slow_time = np.linspace(0, config['CHIRP']['pulse_rep_int']*config['CHIRP']['num_presums']*n_rxs, np.shape(rx_sig_reshaped)[1])
    if duration_s < 0:
        duration_s = np.max(slow_time) - start_time_s
    
    start_idx = np.argmin(np.abs(slow_time - start_time_s))
    end_idx = np.argmin(np.abs(slow_time - (start_time_s + duration_s)))
    actual_duration_s = slow_time[end_idx] - slow_time[start_idx]
    
    rx_sig_cropped = rx_sig_reshaped[:, start_idx:end_idx]
    
    # Coherent stacking
    xcorr_results = np.zeros(((rx_len_samples-len(tx_sig))+1, np.shape(rx_sig_cropped)[1]//n_stack), dtype=np.csingle)
    #print(f"shape of xcorr_results: {np.shape(xcorr_results)}")

    for res_idx in range(np.shape(xcorr_results)[1]):
        stacked = np.mean(rx_sig_cropped[:,res_idx*n_stack:(res_idx+1)*n_stack], axis=1)
        xcorr_results[:, res_idx] = sp.correlate(stacked, tx_sig, mode='valid', method='auto')

        
    fig, axs = plt.subplots(1,1, figsize=(10,5), facecolor='white', sharex=True)
    axs = [axs]
    
    slow_time = np.linspace(start_time_s, start_time_s+actual_duration_s, np.shape(xcorr_results)[1])
    distance_to_reflector = np.linspace(0, np.shape(xcorr_results)[0]/sample_rate, np.shape(xcorr_results)[0]) * sig_speed / 2
    distance_to_reflector = distance_to_reflector - distance_to_reflector[zero_sample_idx]

    p = axs[0].pcolormesh(slow_time, distance_to_reflector, 20*np.log10(np.abs(xcorr_results)), shading='auto')
    #clb = fig.colorbar(p, ax=axs[0])
    #clb.set_label('Power [dB]')
    axs[0].set_xlabel('Time [s]')
    axs[0].set_ylabel('Relative distance to target [m]')
    
    # Plot PX4 log data
    
    if px4_logs is not None:
        df = pd.read_csv(px4_logs + "distance_sensor_0.csv")
        axs[1].plot(((df['timestamp']-np.min(df['timestamp']))*1e-6)-telemetry_time_offset_s, df['current_distance'], label='Laser Rangefinder')
        if overlay_distance_on_radargram:
            axs[0].plot(((df['timestamp']-np.min(df['timestamp']))*1e-6)-telemetry_time_offset_s, df['current_distance'], label='Laser Rangefinder', linestyle=(0, (5,10)))
        df = pd.read_csv(px4_logs + "estimator_global_position_0.csv")
        axs[1].plot(((df['timestamp']-np.min(df['timestamp']))*1e-6)-telemetry_time_offset_s, df['alt']-np.min(df['alt']), label='Fused estimate')
        if overlay_distance_on_radargram:
            axs[0].plot(((df['timestamp']-np.min(df['timestamp']))*1e-6)-telemetry_time_offset_s, df['alt']-np.min(df['alt']), label='Fused estimate', linestyle=(0, (5,10)))
        axs[1].set_xlabel('Time [s]')
        axs[1].set_ylabel('Altitude above ground [m]')
        axs[1].legend()
        axs[1].grid()
        
        # YPR
        df = pd.read_csv(px4_logs + "estimator_attitude_0.csv")
        qx, qy, qz, qw = df['q[0]'], df['q[1]'], df['q[2]'], df['q[3]']

        yaw = np.arctan2(2.0*(qy*qz + qw*qx), qw*qw - qx*qx - qy*qy + qz*qz)
        pitch = -1*np.arcsin(-2.0*(qx*qz - qw*qy))
        roll = np.arctan2(2.0*(qx*qy + qw*qz), qw*qw + qx*qx - qy*qy - qz*qz)

        #axs[2].plot(((df['timestamp']-np.min(df['timestamp']))*1e-6)-telemetry_time_offset_s, yaw*(180/np.pi), label='Yaw')
        axs[2].plot(((df['timestamp']-np.min(df['timestamp']))*1e-6)-telemetry_time_offset_s, pitch*(180/np.pi), label='Pitch')
        axs[2].plot(((df['timestamp']-np.min(df['timestamp']))*1e-6)-telemetry_time_offset_s, roll*(180/np.pi), label='Roll')
        axs[2].set_ylabel('Estimated Angle [deg]')
        #axs[2].legend()
        axs[2].grid()
        
        # Throttle
        ax_throttle = axs[2].twinx()
        df = pd.read_csv(px4_logs + "actuator_controls_0_0.csv")
        ax_throttle.plot(((df['timestamp']-np.min(df['timestamp']))*1e-6)-telemetry_time_offset_s, df['control[3]'], label='Throttle', color='red')
        axs[2].legend()
        
    
    axs[0].set_ylim(300,0)
    #axs[1].set_ylim(0,100)
    axs[0].set_title(f"n_stack = {n_stack}")
    axs[0].set_xlim(start_time_s,start_time_s+duration_s)
    
#     df = pd.read_csv(px4_logs + "actuator_controls_0_0.csv")
#     axs[1].plot(((df['timestamp']-np.min(df['timestamp']))*1e-6)-34, df['control[3]'])

    fig.show()
    return fig, axs

widgets.interact(plot_radargram, n_stack=widgets.IntSlider(min=1, max=300, step=1, value=30), duration_s=widgets.IntSlider(min=-1, max=900, step=1, value=-1), start_time_s=widgets.IntSlider(min=0, max=800, value=0))

In [ ]:
print(f"Peak: {np.max(20*np.log10(np.abs(xcorr_results))):.2f} dB, Median: {np.median(20*np.log10(np.abs(xcorr_results))):.2f} dB")

In [ ]:
20*np.log10(10)

In [ ]:
start_chirp_idx = 69968
min_range = 50
max_range = 150

n_stacks = []
snrs = []
signals = []
noises = []

n_stack = 100

for n_stack in range(1,100):
    stacked = np.mean(rx_sig_reshaped[:,start_chirp_idx:start_chirp_idx+n_stack], axis=1)

    xcorr_res_tmp = sp.correlate(stacked, tx_sig_blackman_window, mode='valid', method='direct')

    slow_time = np.linspace(0, config['CHIRP']['pulse_rep_int']*config['CHIRP']['num_presums']*n_rxs, np.shape(rx_sig_reshaped)[1])

    distance_to_reflector = np.linspace(0, np.shape(xcorr_res_tmp)[0]/sample_rate, np.shape(xcorr_res_tmp)[0]) * sig_speed / 2
    distance_to_reflector = distance_to_reflector - distance_to_reflector[zero_sample_idx]

    min_dist_idx = np.argmin(np.abs(distance_to_reflector - min_range))
    max_dist_idx = np.argmin(np.abs(distance_to_reflector - max_range))

    noise = (np.median(np.abs(xcorr_res_tmp)[min_dist_idx:max_dist_idx]))
    signal = (np.max(np.abs(xcorr_res_tmp)[min_dist_idx:max_dist_idx]))

    n_stacks.append(n_stack)
    snrs.append(signal / noise)
    signals.append(signal)
    noises.append(noise)
    
fig, ax = plt.subplots()
ax.plot(n_stacks, snrs)
#ax.plot(n_stacks, 20*np.log10(signals))
#ax.plot(n_stacks, [1/n for n in noises])
ax.set_aspect('equal')
ax.grid()
#ax.set_ylim(0,5)
#ax.set_xlim(0,5)

In [ ]:
fig, ax = plot_timestep(0, np.minimum(500, np.shape(rx_sig_reshaped)[1]))
#fig.savefig(prefix + "_stacked500_timestep0.png")

In [ ]:
n_stack = 1
start_chirp_idx = 0
stacked = np.mean(rx_sig_reshaped[:,start_chirp_idx:start_chirp_idx+n_stack], axis=1)
xcorr_res_tmp = sp.correlate(stacked, tx_sig_blackman_window, mode='valid', method='direct')

slow_time = np.linspace(0, config['CHIRP']['pulse_rep_int']*config['CHIRP']['num_presums']*n_rxs, np.shape(rx_sig_reshaped)[1])
distance_to_reflector = np.linspace(0, np.shape(xcorr_res_tmp)[0]/sample_rate, np.shape(xcorr_res_tmp)[0]) * sig_speed / 2
distance_to_reflector = distance_to_reflector - distance_to_reflector[zero_sample_idx]

xcorr_res_prev = 20*np.log10(xcorr_res_tmp[(distance_to_reflector > 0) & (distance_to_reflector < 120)])

n_stack_range = range(2,200)
deltas = []

for n_stack in n_stack_range:
    stacked = np.mean(rx_sig_reshaped[:,start_chirp_idx:start_chirp_idx+n_stack], axis=1)
    xcorr_res_tmp = sp.correlate(stacked, tx_sig_blackman_window, mode='valid', method='direct')
    xcorr_cropped = 20*np.log10(xcorr_res_tmp[(distance_to_reflector > 0) & (distance_to_reflector < 120)])
    deltas.append(np.mean(np.abs(xcorr_cropped - xcorr_res_prev)))
    xcorr_res_prev = xcorr_cropped

fig, ax = plt.subplots(figsize=(10,5), facecolor='white')
ax.plot(n_stack_range, deltas)
fig.show()

In [ ]:
config['GENERATE']['chirp_length']
prefix

In [ ]:
prefix
#deltas_500us = copy.deepcopy(deltas)

fig, ax = plt.subplots(figsize=(10,5), facecolor='white')
ax.plot(n_stack_range, deltas_30us, label="30 us")
ax.plot(n_stack_range, deltas_100us, label="100 us")
ax.plot(n_stack_range, deltas_200us, label="200 us")
ax.plot(n_stack_range, deltas_500us, label="500 us")
ax.legend()
ax.set_ylabel('Mean change in power (dB) from n-1 stacking')
ax.set_xlabel('n_stack')
ax.set_xlim(0,20)
fig.show()

In [ ]:
plt.plot(xcorr_res_prev)
plt.show()

In [ ]:
def plot_single_rx_chirp(chirp_idx):
    slow_time = np.linspace(0, config['CHIRP']['pulse_rep_int']*config['CHIRP']['num_presums']*n_rxs, np.shape(rx_sig_reshaped)[1])
    
    fig, axs = plt.subplots(1,2, figsize=(10,4), facecolor='white')
    axs[0].plot(np.real(rx_sig_reshaped[:, chirp_idx]))
    axs[0].set_xlabel('Sample')
    axs[0].set_ylabel('Digitized value')
    axs[0].set_title(f't = {slow_time[chirp_idx]:.2f} s')
    axs[0].set_ylim(-1.1, 1.1)

    f, t, Sxx = sp.spectrogram(rx_sig_reshaped[:, chirp_idx], sample_rate, return_onesided=False)
    axs[1].pcolormesh(t, scipy.fft.fftshift(f)/1e6, scipy.fft.fftshift(Sxx, axes=0), shading='gouraud')
    axs[1].set_ylabel('Frequency [MHz]')
    axs[1].set_xlabel('Time [sec]')
    
    fig.show()
    
    return fig, ax

widgets.interact(plot_single_rx_chirp,
                 chirp_idx=widgets.IntSlider(min=0, max=np.shape(rx_sig_reshaped)[1]-1, step=1, value=0))

In [ ]:
peak_phases = []
peak_idxs = []
for start_chirp_idx in range(500):
    xcorr = sp.correlate(rx_sig_reshaped[:,start_chirp_idx], tx_sig)
    peak_idx = np.argmax(np.abs(xcorr))
    
    peak_idxs.append(peak_idx)
    peak_phases.append(np.angle(xcorr[peak_idx]))
    
fig, ax = plt.subplots(figsize=(10,5), facecolor='white')
line = ax.plot(np.array(peak_phases) * (180.0/np.pi))
mplcursors.cursor(line)
ax.text(0, 1.05, prefix.split("/")[-1] + "\n", horizontalalignment='left', verticalalignment='center', transform=ax.transAxes)
fig.show()

print(np.std(peak_phases))